<a href="https://colab.research.google.com/github/paveethranswam/Speech-style-transfer-and-ASR/blob/main/Wav2Vec2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers  datasets librosa jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.6 MB/s 
     |████████████████████████████████| 451 kB 68.8 MB/s 
     |████████████████████████████████| 7.6 MB 23.2 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 
     |████████████████████████████████| 132 kB 69.1 MB/s 
     |████████████████████████████████| 212 kB 11.0 MB/s 
     |████████████████████████████████| 127 kB 63.8 MB/s 
     |████████████████████████████████| 1.4 MB 42.3 MB/s 
     |████████████████████████████████| 2.2 MB 42.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
!python -m pip install torchaudio > /dev/null
!python -m pip install git+https://github.com/facebookresearch/WavAugment.git > /dev/null

  Running command git clone -q https://github.com/facebookresearch/WavAugment.git /tmp/pip-req-build-pujizpd_


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset, load_metric,Audio
import transformers
import random
import numpy as np
import pandas as pd
import re
from IPython.display import display, HTML
from transformers import AutoProcessor
import librosa
import torch
from datasets import ClassLabel
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_scheduler
import augment
import json
from jiwer import wer
import os
import tqdm
from functools import partial
from IPython.display import display, HTML

In [5]:
# Get device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda:0 device


In [56]:
minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

In [57]:
minds = minds.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [58]:
minds

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 100
})

In [59]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [60]:
show_random_elements(minds.remove_columns(["path", "audio"]))

,transcription
0,I was wondering if my current balance is on my account
1,I would like to know what my current account balance shows
2,how do I set up a joint account
3,show me my account balance
4,can I see my account balance please
5,hello yes my son is going off to college and I'd like to set up a joint account so that emergency
6,how much information about signing up for a joint account
7,how much money is in my account
8,what is my account balance
9,hello I'd like to set up a joint account was my partner how do I do that


In [61]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower()
    return batch

minds = minds.map(remove_special_characters)

In [71]:
vocab_dict = {'vocab': [],'all_text': []}
def extract_all_chars(batch):
  global vocab_dict
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  vocab_dict['vocab'].extend(vocab)
  vocab_dict['all_text'].append(all_text)
  return vocab_dict

m1 = minds.remove_columns(["path", "audio"])
vocabs = m1.map(extract_all_chars)

  0%|          | 0/80 [00:00<?, ?ex/s]

  0%|          | 0/20 [00:00<?, ?ex/s]

In [70]:
vocab_dict['vocab'] = list(set(vocab_dict['vocab']))

In [64]:
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [65]:
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [66]:
minds = minds.train_test_split(test_size=0.2)

In [67]:
minds['train']['audio'][0]['array']

array([-0.00025682, -0.00031486, -0.00023031, ..., -0.00055231,
       -0.00046824, -0.00020242], dtype=float32)

In [68]:
class SpeechTransform:
  def __init__(self, p=0.5):
    self.p = p

  def __call__(self, y):
    if np.random.rand() < self.p:
      return self.apply(y)
    return y

  def apply(self, y: np.ndarray):
      raise NotImplementedError

In [19]:
class ClipEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, rate=0.25):
    super().__init__(p)
    self.rate = rate
    self.sr = sr
  
  def apply(self, x):
    clip_chain = augment.EffectChain().clip(self.rate)
    return clip_chain.apply(x, src_info={'rate': self.sr})

In [20]:
class PitchShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=200):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate

  def apply(self, x):
    rate = np.random.randint(-self.max_rate, self.max_rate)
    return augment.EffectChain().pitch(rate).rate(self.sr).apply(x, src_info={'rate': self.sr})

In [21]:
class ReverbEffect(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_rate=101):
    super().__init__(p)
    self.sr = sr
    self.max_rate = max_rate
  
  def apply(self, x):
    rnd_rate = np.random.randint(0, self.max_rate)
    return augment.EffectChain().reverb(50, 50, rnd_rate).channels(1).apply(x, src_info={'rate': self.sr})

In [22]:
class TimeDropout(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, max_seconds=0.5):
    super().__init__(p)
    self.sr = sr
    self.max_seconds = max_seconds
  
  def apply(self, x):
    return augment.EffectChain().time_dropout(max_seconds=self.max_seconds).apply(x, src_info={'rate': self.sr})

In [23]:
class TimeShift(SpeechTransform):
  def __init__(self, p=0.5, sr=16000, min_shift=0.7, max_shift=2):
    super().__init__(p)
    self.sr = sr
    self.max_shift = max_shift
    self.min_shift = min_shift
  
  def apply(self, x):
    x = x.numpy()
    rate = np.random.uniform(self.min_shift, self.max_shift)
    return torch.as_tensor(librosa.effects.time_stretch(x, rate))

In [24]:
class Compose:
  def __init__(self, transforms):
    self.transforms = transforms
  def __call__(self, x):
    for tr in self.transforms:
      y = tr(x)
    return y

In [25]:
aug_transforms = Compose([ClipEffect(p=0.6),
                          PitchShift(p=0.6),
                          ReverbEffect(p=0.6),
                          TimeDropout(p=0.6),
                          TimeShift(p=0.8)])

In [69]:
def apply_transforms(batch):
      audio = batch["audio"] 
      sp_arr = aug_transforms(torch.Tensor(audio['array']))
      batch['audio']['array'] = sp_arr
      return batch

minds = minds.map(apply_transforms, remove_columns=minds.column_names["train"])

  0%|          | 0/80 [00:00<?, ?ex/s]

  0%|          | 0/20 [00:00<?, ?ex/s]

In [72]:
vocab_list = vocab_dict['vocab']

vocab_list = {v: k for k, v in enumerate(vocab_list)}
vocab_list

{'a': 1709,
 'c': 1712,
 'h': 1719,
 'u': 1720,
 'r': 1722,
 'y': 1708,
 's': 1710,
 ' ': 1713,
 'd': 1716,
 'o': 1717,
 'k': 1683,
 'm': 1703,
 'n': 1714,
 't': 1711,
 'b': 1706,
 'l': 1700,
 'w': 1715,
 'e': 1693,
 'i': 1721,
 'p': 1672,
 "'": 1697,
 'v': 1684,
 'j': 1718,
 'f': 1520,
 'g': 1644}

In [73]:
vocab_list["[UNK]"] = len(vocab_list)
vocab_list["[PAD]"] = len(vocab_list)
print(len(vocab_list))

27


In [74]:
with open('/content/drive/MyDrive/vocab.json', 'w') as vocab_file:
    json.dump(vocab_list, vocab_file)

In [76]:
tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [31]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [32]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [92]:
minds["test"]

Dataset({
    features: ['path', 'audio', 'transcription'],
    num_rows: 20
})

In [34]:
rand_int = random.randint(0, len(minds["train"]))

print("Target text:", minds["train"][rand_int]["transcription"])
print("Input array shape:", np.asarray(minds["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", minds["train"][rand_int]["audio"]["sampling_rate"])

Target text: i would like to be showing my account balance please
Input array shape: (95574,)
Sampling rate: 16000


In [35]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [36]:
minds_tr = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)

#1:   0%|          | 0/20 [00:00<?, ?ex/s]

#0:   0%|          | 0/20 [00:00<?, ?ex/s]

#2:   0%|          | 0/20 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#3:   0%|          | 0/20 [00:00<?, ?ex/s]

...

#0:   0%|          | 0/5 [00:00<?, ?ex/s]

#1:   0%|          | 0/5 [00:00<?, ?ex/s]

#2:   0%|          | 0/5 [00:00<?, ?ex/s]

#3:   0%|          | 0/5 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

In [37]:
def collate_function(batch, processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None):
  b_X = [{"input_values": sample["input_values"]} for sample in batch]
  b_Y = [{"input_ids": sample["labels"]} for sample in batch]
  features = processor.pad(b_X,
            padding=padding,
            max_length=max_length,
            pad_to_multiple_of=pad_to_multiple_of,
            return_tensors="pt") 
  with processor.as_target_processor():
    batchY = processor.pad(b_Y,
        padding=padding,
        max_length=max_length_labels,
        pad_to_multiple_of=pad_to_multiple_of_labels,
        return_tensors="pt")
  labels = batchY["input_ids"].masked_fill(batchY.attention_mask.ne(1), -100)
  features["labels"] = labels
  return features

In [38]:
collate_fn = partial(collate_function, processor=processor, padding=True, max_length=None, 
                     max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None)

In [39]:
version = 2
model_save_path = "/content/drive/MyDrive/wav2vec_finetuning/"
model_name = f"wav2vec_{version}"
batch_size = 8
epochs = 100
lr = 0.0001

In [40]:
train_dataloader = torch.utils.data.DataLoader(minds["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [41]:
batch_sample = next(iter(train_dataloader))

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [42]:
print(batch_sample.keys())

dict_keys(['input_values', 'labels'])


In [43]:
print(batch_sample["input_values"].shape)
print(batch_sample["labels"].shape)

torch.Size([8, 144726])
torch.Size([8, 59])


In [44]:
print(len(train_dataloader))

10


In [78]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes = True
)

Downloading:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.codevectors', 'project_hid.weight', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [79]:
os.makedirs(model_save_path, exist_ok=True)

In [80]:
model.freeze_feature_encoder()

In [81]:
def compute_metrics(labels, preds):
    preds = torch.argmax(preds, axis=-1)
    labels[labels == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(preds)
    label_str = processor.batch_decode(labels, group_tokens=False)
    return wer(label_str, pred_str)

In [82]:
optimizer = AdamW(model.parameters(), lr=lr)

In [83]:
num_training_steps = epochs*len(train_dataloader)
# lr_scheduler = LinearLR(
#     optimizer=optimizer,
#     start_factor=0.5, total_iters=4
# )
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [84]:
scaler = torch.cuda.amp.GradScaler()

In [85]:
model = model.to(device)
model = model.train()

In [86]:
torch.save(model, os.path.join(model_save_path, model_name+".pt"))
processor.save_pretrained(os.path.join(model_save_path, model_name+"_vocab"))

In [87]:
def train_step(train_dataloader, optimizer, lr_scheduler, processor, verbose_at=5, print_sentences=False):
    losses = []
    wers = []
    for step, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        # move inputs to device
        batch = {k: v.to(device) for k, v in data.items()}
        with torch.cuda.amp.autocast():
          outputs = model(**batch)
        loss = outputs.loss
        losses.append(loss.cpu().detach().numpy())
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        preds = outputs.logits
        labels = data["labels"]
        metrics = compute_metrics(labels, preds)
        wers.append(metrics)
        if step%verbose_at == 0:
            print(f"WER: {metrics} \t LOSS: {loss.cpu().detach().numpy()}")
            if print_sentences:
              for l, p in zip(labels, preds):
                  predicted_str = processor.tokenizer.decode(torch.argmax(p, dim =-1))
                  label_str = processor.tokenizer.decode(l)
                  print(f"True: {label_str}; Predicted: {predicted_str}")
    return {"loss": np.mean(losses), "wer":np.mean(wers)}

In [88]:
no_epochs = range(epochs)
epochs_progress_bar = tqdm.tqdm(no_epochs)
curr_best_loss = 1e10
for n in range(epochs):
    res = train_step(train_dataloader, optimizer, lr_scheduler, 
                     processor, verbose_at=len(train_dataloader)//2)
    res["best_loss"] = curr_best_loss
    if curr_best_loss > res["loss"]:
        print("Best model, saving at ",model_save_path)
        torch.save(model, os.path.join(model_save_path, model_name+".pt"))
        curr_best_loss = res["loss"]
        res["best_loss"] = curr_best_loss
    epochs_progress_bar.update(1)

  0%|          | 0/100 [00:30<?, ?it/s]
/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


WER: 1.3846153846153846 	 LOSS: 50.62188720703125
WER: 1.0 	 LOSS: 33.58945846557617
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


  1%|          | 1/100 [00:13<22:59, 13.93s/it]

WER: 1.0 	 LOSS: 29.13821029663086
WER: 1.0 	 LOSS: 5.8065338134765625
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


  2%|▏         | 2/100 [00:28<23:03, 14.12s/it]

...

 98%|█████████▊| 98/100 [20:54<00:26, 13.11s/it]

WER: 1.0 	 LOSS: 2.0229337215423584
WER: 1.0 	 LOSS: 2.2296786308288574


 99%|█████████▉| 99/100 [21:07<00:13, 13.06s/it]

WER: 1.0 	 LOSS: 2.100902795791626
WER: 1.0 	 LOSS: 2.04793643951416
Best model, saving at  /content/drive/MyDrive/wav2vec_finetuning/


100%|██████████| 100/100 [21:26<00:00, 14.78s/it]

In [89]:
model_path = os.path.join(model_save_path, model_name+".pt")
pipeline_path= os.path.join(model_save_path, model_name+"_vocab")

In [90]:
model = model.eval()

In [94]:
minds_te = minds.map(apply_transforms, remove_columns=minds.column_names["test"])
minds_te = minds.map(prepare_dataset, remove_columns=minds_te.column_names["test"], num_proc=4)

#0:   0%|          | 0/20 [00:00<?, ?ex/s]

#1:   0%|          | 0/20 [00:00<?, ?ex/s]

#2:   0%|          | 0/20 [00:00<?, ?ex/s]

#3:   0%|          | 0/20 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

#0:   0%|          | 0/5 [00:00<?, ?ex/s]

#2:   0%|          | 0/5 [00:00<?, ?ex/s]

#1:   0%|          | 0/5 [00:00<?, ?ex/s]

#3:   0%|          | 0/5 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

In [95]:
print(minds_te)

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'input_values', 'labels'],
        num_rows: 80
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'input_values', 'labels'],
        num_rows: 20
    })
})


In [96]:
test_dataloader = torch.utils.data.DataLoader(minds_te["test"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [98]:
next(iter(test_dataloader))['input_values']

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


tensor([[ 1.9871e-04,  1.3853e-04, -3.2478e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 6.1643e-04,  2.7478e-03,  2.5662e-03,  ...,  7.1103e-04,
          1.0655e-04, -2.8897e-04],
        [ 1.0791e-04,  1.5351e-03,  2.6765e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 1.4176e-04,  1.0546e-04, -1.5180e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.2063e-03,  3.7888e-04, -8.0439e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.5085e-05, -1.4290e-04, -2.0794e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

In [99]:
logits = model(next(iter(test_dataloader))['input_values']).logits

RuntimeError: ignored